## Experiments with Behavior Driven Development
This is my introduction to behavior driven development (BDD) in Python. I have implemented a simple RESTful application using the Flask web framework. Lettuce is used enforce the mapping of user stories to tests via very simple description language called gherkin.

References:
- [Behavior Driven Development](https://semaphoreci.com/community/tutorials/behavior-driven-development)
- [Lettuce](http://lettuce.it/)
- [Flask Quick Start](http://flask.pocoo.org/docs/0.10/quickstart/)
- [Designing a RESTful API with Python and Flask](http://blog.miguelgrinberg.com/post/designing-a-restful-api-with-python-and-flask)
- [Best Practices for Designing a Pragmatic RESTful API](http://www.vinaysahni.com/best-practices-for-a-pragmatic-restful-api)

### My first user story written in Gherkin
Gherkin is the language that Lettuce understands. It is a Business Readable, Domain Specific Language that lets one describe software’s behaviour without detailing how that behaviour is implemented.

Lettuce is a Python tool for BDD that understands Gherkin. It can execute plain-text (Gherkin) functional descriptions as automated tests for Python projects.

Lettuce makes the development and testing process traceable, scalable, readable and - what is best - it allows someone who doesn’t program to describe the behavior of our system, without imagining those descriptions will automatically test the system during its development.

In this experiment, user stories are captured in user.features:

In [7]:
%%bash
cat test/features/user.features

Feature: RESTful server
    In order to play with BDD and REST
    As beginners
    We will handle storing, retrieving and deleting user details in a RESTful manner

    Scenario: Retrieve a user's details
        Given some users are in the system
        When I retrieve the user 'david01'
        Then I should get a '200' response
        And the following user details are returned:
            | name |
            | David |
    
    Scenario: Delete an existing user's details
        Given some users are in the system
        When I delete the user 'david01'
        Then I should get a '200' response
        And the message 'User david01 deleted' is returned
        And the user 'david01' is removed
        Then when I retrieve the user 'david01'
        Then I should get a '404' response
        And the message 'User david01 not found' is returned



### Behavior test implementation
User features define behaviors, but Python code is required to test the behaviors.  These are implemented in steps.py.

If one runs lettuce without implementing the step(s), lettuce will quite helpfully provide stubs that will execute but, at this point correctly, fail with a message that they need to be implemented.

In [4]:
%%bash
pygmentize -O style=murphy,linenos=1 ./test/features/steps.py


0001: # -*- coding: utf-8 -*-
0002: 
0003: 
0004: ''
0005: Allow defining steps and store values to be used across each step in th
0006: world object'''
0007: from lettuce import step, world, before
0008: from nose.tools import assert_equals
0009: from app.application import app, USERS
0010: import json
0011: 
0012: # Test user query
0013: @before.all
0014: def before_all():
0015:     '''Setup for testing.'''
0016:     world.app = app.test_client()
0017: 
0018: @step(u'Given some users are in the system')
0019: def given_some_users_are_in_the_system(step):
0020:     '''Add a user to USERS for testing against.'''
0021:     USERS.update({'david01': {'name': 'David'}})
0022: 
0023: @step(u'When I retrieve the user \'(.*)\'')
0024: def when_i_retrieve_the_user_group1(step, username):
0025:     
0026: ''
0027:     A capture group is used in the regular expression allowing us t
0028:     pass in variables to the step. This allows for the reuse of steps    '''
0029:     world.response = worl

### Running the REST server manually with curl
#### GET

In [8]:
%%bash 
curl --silent localhost:5000/

Welcome to my Behavior Driven Development REST server

In [9]:
%%bash 
curl --silent localhost:5000/users

{
  "jack01": {
    "name": "Jack"
  }, 
  "seth02": {
    "name": "Seth"
  }, 
  "zero00": {
    "name": "Zero"
  }
}

In [11]:
%%bash 
curl --silent localhost:5000/user/zero00

{
  "name": "Zero"
}

#### DELETE

In [13]:
%%bash
curl -i --silent -H "Accept: application/json"  http://127.0.0.1:5000/users/zero00

HTTP/1.0 200 OK
Content-Type: application/json
Content-Length: 20
Server: Werkzeug/0.11.2 Python/2.7.6
Date: Mon, 14 Dec 2015 06:18:18 GMT

{
  "name": "Zero"
}

In [15]:
%%bash
curl -i --silent -H "Accept: application/json" -X DELETE http://127.0.0.1:5000/users/zero00

HTTP/1.0 200 OK
Content-Type: text/html; charset=utf-8
Content-Length: 19
Server: Werkzeug/0.11.2 Python/2.7.6
Date: Mon, 14 Dec 2015 06:20:53 GMT

User zero00 deleted

In [16]:
%%bash
curl -i --silent -H "Accept: application/json"  http://127.0.0.1:5000/users/zero00

HTTP/1.0 404 NOT FOUND
Content-Type: text/html; charset=utf-8
Content-Length: 21
Server: Werkzeug/0.11.2 Python/2.7.6
Date: Mon, 14 Dec 2015 06:21:11 GMT

User zero00 not found

### Running the tests

In [17]:
%%bash
lettuce test/features/user.features
# Excuse the double printing. This does not happen on the CLI.


Feature: RESTful server                                                            # test/features/user.features:1
  In order to play with BDD and REST                                               # test/features/user.features:2
  As beginners                                                                     # test/features/user.features:3
  We will handle storing, retrieving and deleting user details in a RESTful manner # test/features/user.features:4

  Scenario: Retrieve a user's details                                              # test/features/user.features:6
    Given some users are in the system                                             # test/features/steps.py:19
    Given some users are in the system                                             # test/features/steps.py:19
    When I retrieve the user 'david01'                                             # test/features/steps.py:24
    When I retrieve the user 'david01'                                             # test/

### TODO
Test:
- Update user data